### Not used to process data

## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import openpyxl

## Access input directories 

In [2]:
#Access input folder
input_dir1 = Path ("OUTPUT_L806")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

# Define current work directory
current_dir = Path.cwd()

1 OUTPUT_L806
2 [WindowsPath('OUTPUT_L806/Output_1'), WindowsPath('OUTPUT_L806/Output_2'), WindowsPath('OUTPUT_L806/Output_3')]


## Filtering arrival time

In [3]:
# Define dataframe for arrival and end time data (Dataframe1)
df = pd.DataFrame(columns = ['File_name', 'Charge_type',
                             'Charge_mass', 'Standoff_distance',
                             'Time_of_arrival', 'End_time'])

# Create output directory
output_dir = current_dir/ "Output_filtered2"
output_dir.mkdir(exist_ok = True)

# Access folders inside input directory
for folder in input_dir2:
    print ("folder", folder)
    
    # Make a list of data file names in each folder
    files = list(folder.rglob("*.xlsx"))
    
    # Access each file inside folders
    for file in files:
        
        # Open excel workbook
        wrkbk = openpyxl.load_workbook(file)
        sh = wrkbk.active
        
        # Define variables to terminate arrival and end time loops
        a=0
        b=0
        
        # Iterate through data to find arrival time
        for i in range(2, sh.max_row+1):
            cell_pressure = sh.cell (row = i, column = 2)
            if cell_pressure.value > 100000:
                print (file.stem)
                print ("pressure", cell_pressure.value)
                arrival_time = sh.cell (row = i, column = 1)
                print ("time of arrival", arrival_time.value)
                a=a+1
            if a == 1:
                break
        print (i)
        
        # Iterate through data to find end time
        for j in range(sh.max_row, 2, -1):
            cell_pressure = sh.cell (row = j, column = 2)
            if cell_pressure.value < 100000:
                print (file.stem)
                print ("pressure", cell_pressure.value)
                end_time = sh.cell (row = j, column = 1)
                print ("end_time", end_time.value)
                b=b+1
            if b == 1:
                break
        print (j)
        
        # Create dataframe to include data points within arrival and end time (Dataframe 2)
        df1 = pd.DataFrame(columns = ['time', 'pressure'])
        df1['time'] = [sh.cell(row=k,column=1).value for k in range(i,j+1)]
        df1['pressure'] = [sh.cell(row=k,column=2).value for k in range(i,j+1)]
        
        # Create folders inside output directory
        output_dir1 = output_dir/ folder.stem
        output_dir1.mkdir(exist_ok = True)
       
        # Create excel files with data points within arrival and end time (Dataframe 2)
        with pd.ExcelWriter(output_dir1/f"{file.stem}.xlsx") as writer:  
            df1.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)
        
        # Append rows to DataFrame 1
        new_row = {'File_name' : [file.stem], 'Charge_type' : ['CompB'], 'Charge_mass' : [3],
                   'Standoff_distance' : [int(file.stem[1:])/2], 'Time_of_arrival' : [arrival_time.value],
                   'End_time' : [end_time.value]}
        new_row_data = pd.DataFrame(new_row)
        df = pd.concat([df, new_row_data], ignore_index = True)


folder OUTPUT_L806\Output_1
T1
pressure 100003
time of arrival 0.0001176500009
1179
T1
pressure 99999.898438
end_time 0.020767999813
70872
T2
pressure 100001
time of arrival 0.00037531999988
3482
T2
pressure 99999.898438
end_time 0.021625999361
73410
T4
pressure 100002
time of arrival 0.0014048999874
7373
T4
pressure 99999.898438
end_time 0.022995999083
77458
T6
pressure 100001
time of arrival 0.0031202998944
14432
T6
pressure 99999.789063
end_time 0.02488199994
83031
T8
pressure 100001
time of arrival 0.0052763000131
22283
T8
pressure 99999.789063
end_time 0.027840999886
91766
folder OUTPUT_L806\Output_2
T10
pressure 100001
time of arrival 0.0076826000586
30201
T10
pressure 99999.898438
end_time 0.030849000439
100648
T12
pressure 100001
time of arrival 0.010239999741
38749
T12
pressure 99999.898438
end_time 0.03387099877
109566
T14
pressure 100001
time of arrival 0.012896000408
47214
T14
pressure 99999.898438
end_time 0.036896001548
118495
T16
pressure 100001
time of arrival 0.0156199

In [4]:
print (df)

   File_name Charge_type Charge_mass  Standoff_distance  Time_of_arrival  \
0         T1       CompB           3                0.5         0.000118   
1         T2       CompB           3                1.0         0.000375   
2         T4       CompB           3                2.0         0.001405   
3         T6       CompB           3                3.0         0.003120   
4         T8       CompB           3                4.0         0.005276   
5        T10       CompB           3                5.0         0.007683   
6        T12       CompB           3                6.0         0.010240   
7        T14       CompB           3                7.0         0.012896   
8        T16       CompB           3                8.0         0.015620   
9        T18       CompB           3                9.0         0.018391   
10       T20       CompB           3               10.0         0.021200   

    End_time  
0   0.020768  
1   0.021626  
2   0.022996  
3   0.024882  
4   0.027841

In [7]:
df2 = df.drop(labels = 0, axis = "index")

In [8]:
print (df1)

   File_name Charge_type Charge_mass  Standoff_distance  Time_of_arrival  \
1         T2       CompB           3                1.0         0.000375   
2         T4       CompB           3                2.0         0.001405   
3         T6       CompB           3                3.0         0.003120   
4         T8       CompB           3                4.0         0.005276   
5        T10       CompB           3                5.0         0.007683   
6        T12       CompB           3                6.0         0.010240   
7        T14       CompB           3                7.0         0.012896   
8        T16       CompB           3                8.0         0.015620   
9        T18       CompB           3                9.0         0.018391   
10       T20       CompB           3               10.0         0.021200   

    End_time  
1   0.021626  
2   0.022996  
3   0.024882  
4   0.027841  
5   0.030849  
6   0.033871  
7   0.036896  
8   0.039922  
9   0.042950  
10  0.045977 

In [9]:
with pd.ExcelWriter('Arrival_time.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)

In [7]:
print (int(file.stem[1:])/2)

4.0
